# Validating Data <a class="anchor" id="top"></a>

In this notebook, you will choose a dataset and prepare it for use with Amazon Personalize.

1. [How to Use the Notebook](#usenotebook)
1. [Introduction](#intro)
1. [Define your Use Case](#usecase)
1. [Choose a Dataset or Data Source](#source)
1. [Prepare the Item Metadata](#prepare_items)
1. [Prepare the Interactions Data](#prepare_interactions)
1. [Prepare the User Metadata](#prepare_users)
1. [Configure an S3 bucket and an IAM  role](#bucket_role)
1. [Create Dataset Group](#group_dataset)
1. [Create the Interactions Schema](#interact_schema)
1. [Create the Items (Movies) Schema](#items_schema)
1. [Create the Users Schema](#users_schema)
1. [Import the Interactions Data](#import_interactions)
1. [Import the Item Metadata](#import_items)
1. [Import the User Metadata](#import_users)

## How to Use the Notebook <a class="anchor" id="usenotebook"></a>

The code is broken up into cells like the one below. There's a triangular Run button at the top of this page that you can click to execute each cell and move onto the next, or you can press `Shift` + `Enter` while in the cell to execute it and move onto the next one.

As a cell is executing you'll notice a line to the side showcase an `*` while the cell is running or it will update to a number to indicate the last cell that completed executing after it has finished exectuting all the code within a cell.

Simply follow the instructions below and execute the cells to get started with Amazon Personalize using case optimized recommenders.

### Some cells are optional

As you work your way through this workshop, some code that is optional will be in cells of the type `Markdown`. To run these cells:
* change the `Cell Type` in the "Cell" menu from `Markdown` to `Code`
* remove the Markdown markers ```` ```Python```` and  ```` ``` ```` before and after the code
* run the cell

## Introduction <a class="anchor" id="intro"></a>
[Back to top](#top)

In Amazon Personalize, you start by creating a dataset group, which is a container for Amazon Personalize components. Your dataset group can be one of the following:

A Domain dataset group, where you create preconfigured resources for different business domains and use cases, such as getting recommendations for similar videos (VIDEO_ON_DEMAND domain) or best selling items (ECOMMERCE domain). You choose your business domain, import your data, and create recommenders. You use recommenders in your application to get recommendations.

Use a [Domain dataset group](https://docs.aws.amazon.com/personalize/latest/dg/domain-dataset-groups.html) if you have a video on demand or e-commerce application and want Amazon Personalize to find the best configurations for your use cases. If you start with a Domain dataset group, you can also add custom resources such as solutions with solution versions trained with recipes for custom use cases.

A [Custom dataset group](https://docs.aws.amazon.com/personalize/latest/dg/custom-dataset-groups.html), where you create configurable resources for custom use cases and batch recommendation workflows. You choose a recipe, train a solution version (model), and deploy the solution version with a campaign. You use a campaign in your application to get recommendations.

Use a Custom dataset group if you don't have a video on demand or e-commerce application or want to configure and manage only custom resources, or want to get recommendations in a batch workflow. If you start with a Custom dataset group, you can't associate it with a domain later. Instead, create a new Domain dataset group.

You can create and manage Domain dataset groups and Custom dataset groups with the AWS console, the AWS Command Line Interface (AWS CLI), or programmatically with the AWS SDKs.


## Define your Use Case <a class="anchor" id="usecase"></a>
[Back to top](#top)

There are a few guidelines for scoping a problem suitable for Personalize. We recommend the values below as a starting point, although the [official limits](https://docs.aws.amazon.com/personalize/latest/dg/limits.html) lie a little lower.

* Authenticated users
* At least 50 unique users
* At least 100 unique items
* At least 2 dozen interactions for each user 

Most of the time this is easily attainable, and if you are low in one category, you can often make up for it by having a larger number in another category.

The user-item-iteraction data is key for getting started with the service. This means we need to look for use cases that generate that kind of data, a few common examples are:

1. Video-on-demand applications
1. E-commerce platforms

Defining your use-case will inform what data and what type of data you need.

In this example we are going to be creating:

1. Amazon Personalize VIDEO_ON_DEMAND Domain recommender for the ["More Like X"](https://docs.aws.amazon.com/personalize/latest/dg/VIDEO_ON_DEMAND-use-cases.html#more-like-y-use-case) use case.
1. Amazon Personalize VIDEO_ON_DEMAND Domain recommender for the ["Top pics for you"](https://docs.aws.amazon.com/personalize/latest/dg/VIDEO_ON_DEMAND-use-cases.html#top-picks-use-case) use case.
1. Amazon Personalize Custom Campaign for a personalized ranked list of movies, for instance shelf/rail/carousel based on some information (director, location, superhero franchise, etc...) 

All of these will be created within the same dataset group and with the same input data.

The diagram bellow shows an overview of what we will be building in this wokshop.

![Workflow](images/image1.png)

In this notebook we will be working on the Data Layer, Creatig a Dataset Group and importing the Datasets. 


## Choose a Dataset or Data Source <a class="anchor" id="source"></a>
[Back to top](#top)

Regardless of the use case, the algorithms all share a base of learning on user-item-interaction data which is defined by 3 core attributes:

1. **UserID** - The user who interacted
1. **ItemID** - The item the user interacted with
1. **Timestamp** - The time at which the interaction occurred

To begin, we are going to use the latest MovieLens dataset, this dataset has over 25 million interactions and a rich collection of metadata for items. There is also a smaller version of this dataset, which can be used to shorten training times, while still incorporating the same capabilities as the full dataset.

Generally speaking your data will not arrive in a perfect form for Personalize, and will take some modification to be structured correctly. This notebook guides you through all of that. 

We will be using data from the [MovieLens project](https://grouplens.org/datasets/movielens/). Follow the link to learn more about the data and potential uses.

First, you will download the dataset from the [MovieLens project](https://grouplens.org/datasets/movielens/) website and unzip it in a new folder using the code below.

In [ ]:
data_dir = "poc_data"
!mkdir $data_dir

!cd $data_dir && wget http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
!cd $data_dir && unzip ml-latest-small.zip
dataset_dir = data_dir + "/ml-latest-small/"

Take a look at the data files you have downloaded.

In [ ]:
!ls $dataset_dir

We can look at the README.txt file

In [ ]:
!pygmentize $data_dir/ml-latest-small/README.txt

## Prepare the Item Metadata <a class="anchor" id="prepare_items"></a>
[Back to top](#top)

Amazon Personalize uses 3 datasets, which usually come from different datasources. 

The item data consists of information about the content that is being interacted with, this generally comes from Content Management Systems (CMS). For the purpose of this workshop we will use the IMDB TT ID to provide a common identifier between the interactions data and the content metadata. Movielens provides its own identifier as well as a the IMDB TT ID (without the leading 'tt') in the 'links.csv' file. We will now convert the interaction data to utilize the IMDB TT id as its unique identifier.

This will allow you to work with filters as well as supporting the [Top Pics for you Domain Recommender](https://docs.aws.amazon.com/personalize/latest/dg/VIDEO_ON_DEMAND-use-cases.html#top-picks-use-case), and complying with the [VIDEO_ON_DEMAND domain dataset and schema requirements](https://docs.aws.amazon.com/personalize/latest/dg/VIDEO-ON-DEMAND-datasets-and-schemas.html#VIDEO-ON-DEMAND-dataset-requirements)..

Python ships with a broad collection of libraries and we need to import those as well as the ones installed to help us like [boto3](https://aws.amazon.com/sdk-for-python/) (AWS SDK for python) and [Pandas](https://pandas.pydata.org/)/[Numpy](https://numpy.org/)  which are core data science tools.

In [ ]:
import time
from time import sleep
import json
from datetime import datetime
import boto3
import pandas as pd
import numpy as np

Next, open the `imdb_items.csv` file and take a look at the first rows. This file has information about the movie.

In [ ]:
original_data = pd.read_csv(data_dir + '/imdbmeta/imdb_items.csv', sep=',', dtype={'PROMOTION': "string"})
original_data.head(5)

In [ ]:
original_data.describe()

This does not really tell us much about the dataset, so we will explore a bit more and look at the raw information. We can see that genres often appear in groups. That is fine for us as Personalize supports this structure.

In [ ]:
original_data.info()

In [ ]:
original_data.isnull().sum()

Looks good, we currently have no null values.

That's it! At this point the item data is ready to go, and we just need to save it as a CSV file.

In [ ]:
items_filename = "item-meta.csv"
original_data.to_csv((data_dir+"/"+items_filename), index=False, float_format='%.0f')

## Prepare the Interactions data <a class="anchor" id="prepare_interactions"></a>
[Back to top](#top)

The next thing to be done is to load the data and confirm the data is in a good state.



Next, open the `ratings.csv` file and take a look at the first rows.

In [ ]:
original_data = pd.read_csv(dataset_dir + '/ratings.csv', sep=',', dtype={'userId': "int64", 'movieId': "str"})
original_data.head(5)

In [ ]:
original_data.shape

In [ ]:
original_data.describe()

This shows that we have a good range of values for `userId` and `movieId`. Next, it is always a good idea to confirm the data format. 

In [ ]:
original_data.info()

In [ ]:
original_data.isnull().any()

In [ ]:
arb_time_stamp = original_data.iloc[50]['timestamp']
print(arb_time_stamp)
print(datetime.utcfromtimestamp(arb_time_stamp).strftime('%Y-%m-%d %H:%M:%S'))

From this, you can see that there are a total of 964982681 entries in the dataset, with 4 columns, userId and timestamp are in int64 format, the rating is a float64 and the movieId is an object.

To use Amazon Personalize, you need to save timestamps in [Unix Epoch](https://en.wikipedia.org/wiki/Unix_time) format.

### Convert the Interactions Data

The interaction data generally is acquired from anaytics platforms that can identify individual interactions with content/items within a platform. 

In [ ]:
links = pd.read_csv(dataset_dir + '/links.csv', sep=',', usecols=[0,1], encoding='latin-1', dtype={'movieId': "str", 'imdbId': "str", 'tmdbId': "str"})
pd.set_option('display.max_rows', 25)
links['imdbId'] = 'tt' + links['imdbId'].astype(object)
links

In [ ]:
imdb_data = original_data.merge(links, on='movieId')
imdb_data.drop(columns='movieId')

Since this is a dataset of an explicit feedback movie ratings, it includes movies rated from 1 to 5. We want to include only moves that were "liked" by the users, and simulate a dataset of data that would be gathered by a VOD platform. In order to do that, we will filter out all interactions under 2 out of 5, and create two event types: "Click" and and "Watch". We will then assign all movies rated 2 and above as "Click" and movies rated 4 and above as both "Click" and "Watch". 

Note that for a real data set you would actually model based on implicit feedback such as clicks, watches and/or explicit feedback such as ratings, likes etc.

In [ ]:
watched_df = imdb_data.copy()
watched_df = watched_df[watched_df['rating'] > 3]
watched_df = watched_df[['userId', 'imdbId', 'timestamp']]
watched_df['EVENT_TYPE']='Watch'
watched_df.head()

In [ ]:
clicked_df = imdb_data.copy()
clicked_df = clicked_df[clicked_df['rating'] > 1]
clicked_df = clicked_df[['userId', 'imdbId', 'timestamp']]
clicked_df['EVENT_TYPE']='Click'
clicked_df.head()

In [ ]:
interactions_df = clicked_df.copy()
interactions_df = interactions_df.append(watched_df)
interactions_df.sort_values("timestamp", axis = 0, ascending = True, 
                 inplace = True, na_position ='last') 

In [ ]:
interactions_df.info()

Lets look at what the new dataset looks like.

In [ ]:
interactions_df.describe()

 Amazon Personalize has default column names for users, items, and timestamp. These default column names are `USER_ID`, `ITEM_ID`, `TIMESTAMP` and `EVENT_VALUE` for the [VIDEO_ON_DEMAND domain dataset](https://docs.aws.amazon.com/personalize/latest/dg/VIDEO-ON-DEMAND-datasets-and-schemas.html). The final modification to the dataset is to replace the existing column headers with the default headers.

In [ ]:
interactions_df.rename(columns = {'userId':'USER_ID', 'imdbId':'ITEM_ID', 
                              'timestamp':'TIMESTAMP'}, inplace = True) 

We'll be using a subset of the IMDB dataset for this workshop that has been cleaned to remove movies that don't have valid values for the metadata we are using in out ITEMs dataset (we'll work with this more in the net section), so we'll need to make sure we don't have any interactions that have IMDB movie ids that are not in our subset of the IMDB data set.

In [ ]:
movies = pd.read_csv(data_dir + '/imdbmeta' + '/imdb_items.csv', sep=',', usecols=[0,1], encoding='latin-1', dtype={'movieId': "str", 'imdbId': "str", 'tmdbId': "str"})
pd.set_option('display.max_rows', 25)

Next, let's compare the number of ITEM_ID unique keys in the IMDB data to the ITEM_ID unique keys in the interactions.  They should be the same.

In [ ]:
movies.nunique(axis=0)

In [ ]:
interactions_df.nunique(axis=0)

The number of unique ITEM_IDs are not the same in the IMDB data and the interactions data, so we'll clean out the data points with ITEM_IDs that do not have item metadata from the interactions dataset.

In [ ]:
interactions_df = interactions_df.merge(movies, on='ITEM_ID')
interactions_df.info()

We will also drop the `TITLE` column as it is not required in the interactions dataset.

In [ ]:
interactions_df = interactions_df.drop(columns=['TITLE'])
interactions_df.info()

That's it! At this point the data is ready to go, and we just need to save it as a CSV file.

In [ ]:
interactions_filename = "interactions.csv"
interactions_df.to_csv((data_dir+"/"+interactions_filename), index=False, float_format='%.0f')

## Prepare the User Metadata <a class="anchor" id="prepare_users"></a>
[Back to top](#top)

This will supporting the [Top Pics for you Domain Recommender](https://docs.aws.amazon.com/personalize/latest/dg/VIDEO_ON_DEMAND-use-cases.html#top-picks-use-case), and complying with the [VIDEO_ON_DEMAND domain dataset and schema requirements](https://docs.aws.amazon.com/personalize/latest/dg/VIDEO-ON-DEMAND-datasets-and-schemas.html#VIDEO-ON-DEMAND-dataset-requirements).

The dataset does not have any user metadata so we will create a fake metadata field.

In [ ]:
# get all unique user ids from the interaction dataset

user_ids = interactions_df['USER_ID'].unique()
user_data = pd.DataFrame()
user_data["USER_ID"]= user_ids
user_data

### Adding User Metadata

The current dataset does not contain additiona user information. For this example, we'll randomly assign a membership level. For Ad Supported models this could indicate premium vs ad supported.

In [ ]:
possible_membership_levels = ['silver', 'gold']
random = np.random.choice(possible_membership_levels, len(user_data.index), p=[0.5, 0.5])
user_data["MEMBERLEVEL"] = random
user_data

In [ ]:
# Saving the data as a CSV file
users_filename = "users.csv"
user_data.to_csv((data_dir+"/"+users_filename), index=False, float_format='%.0f')


We will use this data in the next labs. In order to use this data we will store these variables so subsequent notebooks can use this data. 

In [ ]:
%store dataset_dir
%store data_dir
%store interactions_filename
%store items_filename
%store users_filename

[Go to the next notebook `02_Training_Layer_Recap.ipynb`](02_Training_Layer_Recap.ipynb) to continue.